### GOLD LAYER LOAD

**Purpose:**  
This notebook performs the **Gold layer aggregation** for farm advisory, combining multiple Silver-layer datasets (soil, crop, market, pest, rainfall) into a single enriched dataset for downstream analytics and AI insights.

**Workflow:**  
1. Load Silver tables from the Delta Lake (soil, crop, market, pest, rainfall).  
2. Rename columns and cast numeric fields for consistency.  
3. Join datasets on **date** and **city** to create a unified Gold table.  
4. Compute derived business metrics:
   - `yieldPredictionScore`  
   - `profitabilityIndex`  
   - `pestRiskCategory`  
   - `soilMoistureCategory`  
   - `sustainabilityScore`  
   - `description` (text summary for each crop-city-date)
5. Save the resulting Gold table to Delta format for analytics and AI workflows.  
6. Write audit logs for each task (success/failure) to an audit Delta table.

**Dashboard & Visualization:**  
- Visualize **yield trends** by city and crop.  
- View **profitability ranking** of top crops.  
- Heatmaps for **monthly crop price trends**.  
- Supports interactive plots using **Matplotlib** and **Seaborn** in the same notebook.

**Notes:**  
- Update the variables `catalog_name`, `schema_name_silver`, `schema_name_gold`, `audit_schema_name`, and `workflow_job_id` to match your Databricks environment.  
- Ensure the cluster has sufficient memory for joining multiple Silver tables and computing derived metrics.  
- This notebook is designed to feed downstream AI and vector-based query systems for farm advisory

In [0]:
import traceback
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, when, lit, concat_ws, current_timestamp
)
from pyspark.sql.types import DoubleType
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, current_timestamp
import numpy as np
import pandas as pd

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS databricks_free_edition.databricks_gold;

In [0]:
catalog_name = "databricks_free_edition"
schema_name = "databricks_gold"
schema_name_silver = "databricks_silver"
audit_schema_name = "audit_logs"
schema_name_gold = "databricks_gold"
workflow_job_id = dbutils.widgets.get("workflow_job_id")


In [0]:


db_silver = f"{catalog_name}.{schema_name_silver}"
db_gold = f"{catalog_name}.{schema_name_gold}"
db_audit = f"{catalog_name}.{audit_schema_name}"

# -----------------------------
# Audit Log Function
# -----------------------------

def write_audit(workflow_job_id,task, status, start_ts, end_ts, message=""):
    """Write task audit record."""
    rows = [(workflow_job_id,task, status, start_ts, end_ts, message)]
    schema = "workflow_job_id STRING, task STRING, status STRING, start_time TIMESTAMP, end_time TIMESTAMP, message STRING"
    spark.createDataFrame(rows, schema=schema) \
        .write.format("delta") \
        .mode("append") \
        .saveAsTable(f"{db_audit}.error_reporting_audit")

# -----------------------------
# GOLD AGGREGATION
# -----------------------------
task_name_gold = "gold_layer_farm_aggregation"
start_ts_gold = spark.sql("SELECT current_timestamp()").collect()[0][0]

try:
    print("🚜 Starting Gold layer aggregation...")

    # -----------------------------
    # Helper: rename and cast numeric columns
    # -----------------------------
    def rename_and_cast(df, prefix):
        for c in df.columns:
            df = df.withColumnRenamed(c, f"{prefix}_{c}")
        num_cols = [
            "soilMoisture", "temperature", "humidity", "precipitationIntensity",
            "cropHealthScore", "ndviIndex", "leafMoisture", "cropPricePerQuintal", "pestRisk", "rainfall"
        ]
        for c in num_cols:
            if f"{prefix}_{c}" in df.columns:
                df = df.withColumn(f"{prefix}_{c}", col(f"{prefix}_{c}").cast(DoubleType()))
        return df

    # -----------------------------
    # Load Silver Tables
    # -----------------------------
    soil = rename_and_cast(spark.table(f"{db_silver}.silver_soil"), "soil")
    crop = rename_and_cast(spark.table(f"{db_silver}.silver_crop"), "crop")
    market = rename_and_cast(spark.table(f"{db_silver}.silver_market"), "market")
    pest = rename_and_cast(spark.table(f"{db_silver}.silver_pest"), "pest")
    rainfall = rename_and_cast(spark.table(f"{db_silver}.silver_rainfall"), "rainfall")

    # -----------------------------
    # Join Datasets on date + city
    # -----------------------------
    gold = (
        soil.join(crop, (soil["soil_date"] == crop["crop_date"]) & (soil["soil_city"] == crop["crop_city"]), "inner")
            .join(market, (soil["soil_date"] == market["market_date"]) & (soil["soil_city"] == market["market_city"]), "inner")
            .join(pest, (soil["soil_date"] == pest["pest_date"]) & (soil["soil_city"] == pest["pest_city"]), "inner")
            .join(rainfall, (soil["soil_date"] == rainfall["rainfall_date"]) & (soil["soil_city"] == rainfall["rainfall_city"]), "inner")
            .drop("crop_date","market_date","pest_date","rainfall_date",
                  "crop_city","market_city","pest_city","rainfall_city")
            .withColumnRenamed("soil_date","date")
            .withColumnRenamed("soil_city","city")
    )

    # -----------------------------
    # Business Metrics / Derived Columns
    # -----------------------------
    gold = (
        gold
        .withColumn("yieldPredictionScore", col("soil_soilMoisture") * col("crop_cropHealthScore") / 100)
        .withColumn("profitabilityIndex", col("yieldPredictionScore") * col("crop_cropPricePerQuintal") / 100)
        .withColumn("pestRiskCategory",
                    when(col("pest_pestRisk") > 70, "High")
                    .when(col("pest_pestRisk") > 40, "Medium")
                    .otherwise("Low"))
        .withColumn("soilMoistureCategory",
                    when(col("soil_soilMoisture") < 30, "Dry")
                    .when(col("soil_soilMoisture") < 60, "Normal")
                    .otherwise("Wet"))
        .withColumn("sustainabilityScore",
                    (col("yieldPredictionScore") - col("pest_pestRisk")) / 10)
        .withColumn("description",
                    concat_ws(" ",
                        lit("Crop:"), col("market_cropName"),
                        lit("in"), col("city"),
                        lit("has soil moisture"), col("soil_soilMoisture"),
                        lit("pest risk"), col("pestRiskCategory"),
                        lit("yield score"), col("yieldPredictionScore"),
                        lit("profitability"), col("profitabilityIndex"),
                        lit("rainfall(mm)"), col("rainfall_rainfallMm")
                    ))
    )

    # -----------------------------
    # Save Gold Table
    # -----------------------------
    gold_table = f"{db_gold}.gold_farm_advisor"
    spark.sql(f"DROP TABLE IF EXISTS {gold_table}")
    gold.write.format("delta").mode("overwrite").saveAsTable(gold_table)

    end_ts_gold = spark.sql("SELECT current_timestamp()").collect()[0][0]
    write_audit(workflow_job_id, task_name_gold, "SUCCESS", start_ts_gold, end_ts_gold, "Gold table created")
    print(f"✅ Gold table created successfully: {gold_table}")

except Exception as e:
    end_ts_gold = spark.sql("SELECT current_timestamp()").collect()[0][0]
    tb = traceback.format_exc()
    write_audit(workflow_job_id, task_name_gold, "FAILED", start_ts_gold, end_ts_gold, str(tb)[:4000])
    print("❌ Gold aggregation failed. See audit table for details.")
    raise


### Yield trend by City

In [0]:
df = spark.table(gold_table).toPandas()

plt.figure(figsize=(12,6))
sns.lineplot(x="date", y="yieldPredictionScore", hue="city", data=df)
plt.title("Yield Prediction Trend by City")
plt.xlabel("Date")
plt.ylabel("Yield Score")
plt.xticks(rotation=45)
display(plt.gcf())  


### Top 10 profitable crops

In [0]:
top_profit = (
    df.groupby("market_cropName")["profitabilityIndex"]
      .mean()
      .reset_index()
      .sort_values("profitabilityIndex", ascending=False)
      .head(10)
)

plt.figure(figsize=(12,6))
sns.barplot(x="profitabilityIndex", y="market_cropName", data=top_profit)
plt.title("Top 10 Most Profitable Crops")
plt.xlabel("Profitability Index")
plt.ylabel("Crop")
display(plt.gcf())  


###  --- PRICE TREND HEATMAP ---

In [0]:
# Ensure date is datetime
df["date"] = pd.to_datetime(df["date"])

# Extract month name
df["month"] = df["date"].dt.strftime("%b")

# Pivot table: Rows = Crop, Columns = Month, Values = Avg Price
heatmap_df = (
    df.groupby(["market_cropName", "month"])["crop_cropPricePerQuintal"]
      .mean()
      .reset_index()
      .pivot(index="market_cropName", columns="month", values="crop_cropPricePerQuintal")
)

# Sort months properly (instead of alphabetical)
month_order = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
heatmap_df = heatmap_df.reindex(columns=[m for m in month_order if m in heatmap_df.columns])

# Plot Heatmap
plt.figure(figsize=(14,10))
sns.heatmap(heatmap_df, annot=True, fmt=".1f", linewidths=.5)
plt.title("📈 Average Crop Price Trend Heatmap (₹ per Quintal)")
plt.xlabel("Month")
plt.ylabel("Crop")

display(plt.gcf())  
